In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("../dataset/shadowfax_mixed_logistics_dataset.csv")
df.head()


,order_id,order_type,zone,hub_id,rider_id,order_date,expected_delivery_time,actual_delivery_time,distance_km,base_cost,rider_payment
0,1,e-commerce,Zone D,Hub_4,R1096,2025-01-10 13:45:00,2025-01-10 15:09:00,2025-01-10 15:08:20.300602,11.51,76.30,104.31
1,2,courier,Zone A,Hub_3,R1124,2025-02-21 15:37:00,2025-02-21 17:42:00,2025-02-21 17:01:48.936374,3.12,20.68,28.27
2,3,food,Zone A,Hub_20,R1035,2025-02-19 13:24:00,2025-02-19 15:33:00,2025-02-19 15:59:37.077638,8.12,53.83,73.59
3,4,food,Zone D,Hub_6,R1099,2025-02-17 13:13:00,2025-02-17 14:21:00,2025-02-17 14:09:03.712500,6.37,42.23,57.73
4,5,e-commerce,Zone C,Hub_15,R1023,2025-01-12 06:06:00,2025-01-12 07:33:00,2025-01-12 07:29:15.715270,8.21,54.42,74.40


In [4]:
df['order_date'] = pd.to_datetime(df['order_date'])
df['expected_delivery_time'] = pd.to_datetime(df['expected_delivery_time'])
df['actual_delivery_time'] = pd.to_datetime(df['actual_delivery_time'])


In [5]:
df['delay_mins'] = (df['actual_delivery_time'] - df['expected_delivery_time']).dt.total_seconds() / 60


In [6]:
df['sla_met'] = df['delay_mins'].apply(lambda x: 1 if x <= 0 else 0)


In [7]:
def delay_bucket(x):
    if x <= 0:
        return "On Time / Early"
    elif x <= 10:
        return "0–10 mins delay"
    elif x <= 30:
        return "10–30 mins delay"
    elif x <= 60:
        return "30–60 mins delay"
    else:
        return "60+ mins delay"

df['delay_bucket'] = df['delay_mins'].apply(delay_bucket)


In [8]:
df['cost_leakage'] = df['rider_payment'] - df['base_cost']
df['cost_leakage_flag'] = df['cost_leakage'].apply(lambda x: 1 if x > 0 else 0)


In [9]:
df.to_csv("../dataset/cleaned_deliveries.csv", index=False)


In [10]:
df.head()
df.describe()
df['delay_bucket'].value_counts()


delay_bucket
On Time / Early     20046
10–30 mins delay     9648
0–10 mins delay      7687
30–60 mins delay     2566
60+ mins delay         53
Name: count, dtype: int64

In [11]:
import pandas as pd

df = pd.read_csv("../dataset/cleaned_deliveries.csv")
df.head()


,order_id,order_type,zone,hub_id,rider_id,order_date,expected_delivery_time,actual_delivery_time,distance_km,base_cost,rider_payment,delay_mins,sla_met,delay_bucket,cost_leakage,cost_leakage_flag
0,1,e-commerce,Zone D,Hub_4,R1096,2025-01-10 13:45:00,2025-01-10 15:09:00,2025-01-10 15:08:20.300602,11.51,76.30,104.31,-0.661657,1,On Time / Early,28.01,1
1,2,courier,Zone A,Hub_3,R1124,2025-02-21 15:37:00,2025-02-21 17:42:00,2025-02-21 17:01:48.936374,3.12,20.68,28.27,-40.184394,1,On Time / Early,7.59,1
2,3,food,Zone A,Hub_20,R1035,2025-02-19 13:24:00,2025-02-19 15:33:00,2025-02-19 15:59:37.077638,8.12,53.83,73.59,26.617961,0,10–30 mins delay,19.76,1
3,4,food,Zone D,Hub_6,R1099,2025-02-17 13:13:00,2025-02-17 14:21:00,2025-02-17 14:09:03.712500,6.37,42.23,57.73,-11.938125,1,On Time / Early,15.50,1
4,5,e-commerce,Zone C,Hub_15,R1023,2025-01-12 06:06:00,2025-01-12 07:33:00,2025-01-12 07:29:15.715270,8.21,54.42,74.40,-3.738079,1,On Time / Early,19.98,1


In [12]:
total_orders = len(df)
avg_delay = df['delay_mins'].mean()
sla_percent = df['sla_met'].mean() * 100
leakage_orders = df['cost_leakage_flag'].sum()
leakage_total = df.loc[df['cost_leakage_flag'] == 1, 'cost_leakage'].sum()

print(f"Total orders          : {total_orders}")
print(f"Average delay (mins)  : {avg_delay:.2f}")
print(f"SLA met (%)           : {sla_percent:.2f}%")
print(f"Leakage orders        : {leakage_orders}")
print(f"Total cost leakage    : {leakage_total:.2f}")


Total orders          : 40000
Average delay (mins)  : -0.10
SLA met (%)           : 50.11%
Leakage orders        : 40000
Total cost leakage    : 752266.03


In [13]:
order_type_summary = (
    df.groupby('order_type')
      .agg(
          total_orders=('order_id', 'count'),
          avg_delay_mins=('delay_mins', 'mean'),
          sla_percent=('sla_met', lambda x: x.mean() * 100)
      )
      .round(2)
      .sort_values('avg_delay_mins', ascending=False)
)

order_type_summary


,total_orders,avg_delay_mins,sla_percent
order_type,,,
courier,7946,0.21,49.16
food,12064,-0.14,50.55
e-commerce,19990,-0.21,50.24


In [14]:
zone_summary = (
    df.groupby('zone')
      .agg(
          total_orders=('order_id', 'count'),
          avg_delay_mins=('delay_mins', 'mean'),
          sla_percent=('sla_met', lambda x: x.mean() * 100)
      )
      .round(2)
      .sort_values('avg_delay_mins', ascending=False)
)

zone_summary


,total_orders,avg_delay_mins,sla_percent
zone,,,
Zone C,9905,0.25,49.06
Zone A,10041,-0.07,50.14
Zone D,10019,-0.14,50.05
Zone B,10035,-0.45,51.19


In [15]:
hub_summary = (
    df.groupby('hub_id')
      .agg(
          total_orders=('order_id', 'count'),
          avg_delay_mins=('delay_mins', 'mean'),
          sla_percent=('sla_met', lambda x: x.mean() * 100)
      )
      .round(2)
      .sort_values('sla_percent')  # worst first
)

hub_summary.head()


,total_orders,avg_delay_mins,sla_percent
hub_id,,,
Hub_2,1985,0.77,47.51
Hub_8,2021,0.67,48.19
Hub_3,1928,0.40,48.34
Hub_17,1933,0.34,48.99
Hub_11,2024,0.07,49.11


In [16]:
rider_summary = (
    df.groupby('rider_id')
      .agg(
          total_deliveries=('order_id', 'count'),
          avg_delay_mins=('delay_mins', 'mean'),
          sla_percent=('sla_met', lambda x: x.mean() * 100)
      )
      .query("total_deliveries >= 50")   # filter to riders with enough data
      .round(2)
      .sort_values('sla_percent', ascending=False)
)

rider_summary.head()


,total_deliveries,avg_delay_mins,sla_percent
rider_id,,,
R1144,215,-3.23,58.14
R1026,211,-2.93,57.82
R1043,215,-2.17,57.21
R1051,188,-2.46,56.91
R1106,201,-0.74,56.72


In [17]:
delay_bucket_summary = (
    df['delay_bucket']
      .value_counts()
      .rename_axis('delay_bucket')
      .reset_index(name='order_count')
)

delay_bucket_summary


,delay_bucket,order_count
0,On Time / Early,20046
1,10–30 mins delay,9648
2,0–10 mins delay,7687
3,30–60 mins delay,2566
4,60+ mins delay,53
